In [8]:
import pandas as pd
import torch
import torch.nn.functional as F
import numpy as np
import torch.nn as nn
import time
from datetime import timedelta
import copy
from torch.nn import TransformerEncoder, TransformerEncoderLayer
from sklearn import metrics
from tensorboardX import SummaryWriter
from sklearn.utils import shuffle
df1=pd.read_csv('./another/811/train.csv',sep=',')
df2=pd.read_csv('./another/811/val.csv',sep=',')
df3=pd.read_csv('./another/811/test.csv',sep=',')
df1 = df1.reset_index(drop=True)
df2 = df2.reset_index(drop=True)
df3 = df3.reset_index(drop=True)
df1=shuffle(df1)
df2=shuffle(df2)
df3=shuffle(df3)
df1=df1.iloc[:,1:]
df2=df2.iloc[:,1:]
df3=df3.iloc[:,1:]
df1.iloc[:,293:-16]=0
df2.iloc[:,293:-16]=0
df3.iloc[:,293:-16]=0
df1=df1.fillna(0)
df2=df2.fillna(0)
df3=df3.fillna(0)
arr1=np.array(df1.iloc[:,:-1])
arr2=np.array(df2.iloc[:,:-1])
arr3=np.array(df3.iloc[:,:-1])
arr1.shape,arr2.shape
def ts(x,pad_size,dim_model):
        t1=x.reshape(x.shape[0],pad_size,dim_model+2*pad_size+1)
        tdag1=t1[:,:,dim_model+1:dim_model+1+pad_size]
        tdag2=t1[:,:,dim_model+1+pad_size:]
        tdag=tdag1+tdag2
        for x in range(x.shape[0]):
            for y in range(pad_size):
                tdag[x][y][y]=1
        tdata=t1[:,:,:dim_model]
        return tdata,tdag,tdag2,tdag1
data1,mask1,dagin1,dagout1=ts(arr1,7,34)
data2,mask2,dagin2,dagout2=ts(arr2,7,34)
data3,mask3,dagin3,dagout3=ts(arr3,7,34)
data1.shape,data2.shape
def searchdag(dagin):
    pos=np.zeros(dagin.shape[1])
    for x in range(dagin.shape[0]):
        if(dagin[x].sum()==0):
            pos[x]=0
        else:
            listnode=[]
            for y in range(dagin.shape[1]):
                if(dagin[x][y]!=0):
                    listnode.append(y)
            #Flag=True
            n=0
            listmax=[]
            
            for z in listnode:
                n=0
                list1=[]
                if(dagin[z].sum()!=0):
                    for p in range(dagin.shape[1]):
                        #n=0
                        if(dagin[z][p]!=0):
                            list1.append(p)
                            n+=1
                        while(list1!=[]):
                            listnode=[]
                            for z in list1:
                                #del listnode[z]
                                
                                if(dagin[z].sum()!=0):
                                    for p in range(dagin.shape[1]):
                                        if(dagin[z][p]!=0):
                                            listnode.append(p)
                                            n+=1
                            list1=listnode
                    else:
                        n+=1
                    listmax.append(n)
                            
                            
                            
                else:
                    n+=1
                    listmax.append(n)
                
            
                
            pos[x]=np.array(listmax).max(0)
    return pos
pos1=np.zeros((arr1.shape[0],7))
for x in range(pos1.shape[0]):
    pos1[x]=searchdag(dagin1[x])
pos2=np.zeros((arr2.shape[0],7))
for x in range(pos2.shape[0]):
    pos2[x]=searchdag(dagin2[x])
pos3=np.zeros((arr3.shape[0],7))
for x in range(pos3.shape[0]):
    pos3[x]=searchdag(dagin3[x])
import torch

class Config(object):

    def __init__(self):
        self.model_name = 'Transformer'
              
        self.log_path = './modellog/' + self.model_name
        
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')   # 设备

        self.dropout = 0.5                                              
        self.require_improvement = 2000                                
        self.num_classes = 3                                           
        
        self.num_epochs = 1000                                          
        self.batch_size = 500                                         
        self.pad_size = 7                                             
        self.learning_rate = 1e-4                                      
        self.embed = 34         
        self.dim_model = 34
        self.hidden = 1024
        self.last_hidden = 512
        self.num_head = 8
        self.num_encoder = 6
        self.txdim=512
        self.res_num_layer=4
config=Config()
train_data=[]
def create_position(pos,pad_size,embed):
    
    pe = np.array([[[posit / (10000.0 ** (i // 2 * 2.0 / embed))  for i in range(embed)]for posit in posi] for posi in pos])
    pe[:,:,0::2]=np.sin(pe[:,:,0::2])
    pe[:,:,1::2]=np.cos(pe[:,:,1::2])
    return pe
position1=create_position(pos1,config.pad_size,config.embed)
data=np.vstack((data1.reshape(data1.shape[0]*7,34),data2.reshape(data2.shape[0]*7,34),data3.reshape(data3.shape[0]*7,34)))
data=(data-data.min(0))/(data.max(0)-data.min(0))
data[np.isnan(data)]=0
data1=data[:data1.shape[0]*7,:].reshape(data1.shape[0],7,34)
data2=data[data1.shape[0]*7:data1.shape[0]*7+data2.shape[0]*7,:].reshape(data2.shape[0],7,34)
data3=data[data1.shape[0]*7+data2.shape[0]*7:,:].reshape(data3.shape[0],7,34)
data1[:,-1,:]=0
data2[:,-1,:]=0
data3[:,-1,:]=0
for x in range(arr1.shape[0]):
    train_data.append((data1[x],df1.iloc[x,-1],position1[x],mask1[x]))
position2=create_position(pos2,config.pad_size,config.embed)
position3=create_position(pos3,config.pad_size,config.embed)

dev_data=[]
test_data=[]
for x in range(arr2.shape[0]):
    dev_data.append((data2[x],df2.iloc[x,-1],position2[x],mask2[x]))
for x in range(arr3.shape[0]):
    test_data.append((data3[x],df3.iloc[x,-1],position3[x],mask3[x]))
class DatasetIterater(object):
    def __init__(self, batches, batch_size, device):
        self.batch_size = batch_size
        self.batches = batches
        self.n_batches = len(batches) // batch_size
        self.residue = False  
        if len(batches) % self.n_batches != 0:
            self.residue = True
        self.index = 0
        self.device = device

    def _to_tensor(self, datas):
        x = torch.FloatTensor([_[0] for _ in datas]).to(self.device)
        y = torch.LongTensor([_[1] for _ in datas]).to(self.device)
        z = torch.FloatTensor([_[2] for _ in datas]).to(self.device)
        m = torch.FloatTensor([_[3] for _ in datas]).to(self.device)
        m = m.masked_fill(m == 0, float('-inf')).masked_fill(m != 0, float(0.0))


        return (x), y,z,m

    def __next__(self):
        if self.residue and self.index == self.n_batches:
            batches = self.batches[self.index * self.batch_size: len(self.batches)]
            self.index += 1
            batches = self._to_tensor(batches)
            return batches

        elif self.index >= self.n_batches:
            self.index = 0
            raise StopIteration
        else:
            batches = self.batches[self.index * self.batch_size: (self.index + 1) * self.batch_size]
            self.index += 1
            batches = self._to_tensor(batches)
            return batches

    def __iter__(self):
        return self

    def __len__(self):
        if self.residue:
            return self.n_batches + 1
        else:
            return self.n_batches


def build_iterator(dataset, config):
    iter = DatasetIterater(dataset, config.batch_size, config.device)
    return iter


def get_time_dif(start_time):
    """获取已使用时间"""
    end_time = time.time()
    time_dif = end_time - start_time
    return timedelta(seconds=int(round(time_dif)))
import pickle
train_iter = build_iterator(train_data, config)
dev_iter = build_iterator(dev_data, config)
test_iter = build_iterator(test_data, config)
import numpy as np
import pandas as pd 
import torch
import copy
from torch.nn import functional as F
import torch.nn as nn
from torch.nn.modules import Module

#from .container import ModuleList
from torch.nn.modules.dropout import Dropout
from torch.nn.modules.linear import Linear
from torch.nn.modules.normalization import LayerNorm
from torch.nn.init import xavier_uniform_
from torch.nn.init import constant_
from torch.nn.init import xavier_normal_
from torch.nn.parameter import Parameter
from torch.nn.functional import linear
from torch.nn.functional import softmax
from torch.nn.functional import dropout


import pandas as pd
import torch
import torch.nn.functional as F
import numpy as np
import torch.nn as nn
import time
from datetime import timedelta
import copy
from torch.nn import TransformerEncoder, TransformerEncoderLayer
from sklearn import metrics
from tensorboardX import SummaryWriter
from sklearn.utils import shuffle


import numpy as np
import pandas as pd 
import torch
import copy
from torch.nn import functional as F
import torch.nn as nn
from torch.nn.modules import Module

#from .container import ModuleList
from torch.nn.modules.dropout import Dropout
from torch.nn.modules.linear import Linear
from torch.nn.modules.normalization import LayerNorm
from torch.nn.init import xavier_uniform_
from torch.nn.init import constant_
from torch.nn.init import xavier_normal_
from torch.nn.parameter import Parameter
from torch.nn.functional import linear
from torch.nn.functional import softmax
from torch.nn.functional import dropout


def _get_activation_fn(activation):
    if activation == "relu":
        return F.relu
    elif activation == "gelu":
        return F.gelu
    else:
        raise RuntimeError("activation should be relu/gelu, not %s." % activation)


class DAGMultiheadAttention(Module):
    
    __annotations__ = {#注解
        'bias_k': torch._jit_internal.Optional[torch.Tensor],
        'bias_v': torch._jit_internal.Optional[torch.Tensor],
    }
    __constants__ = ['q_proj_weight', 'k_proj_weight', 'v_proj_weight', 'in_proj_weight']

    def __init__(self, embed_dim, num_heads, dropout=0., bias=True, add_bias_kv=False, add_zero_attn=False, kdim=None, vdim=None):
        super(DAGMultiheadAttention, self).__init__()
        self.embed_dim = embed_dim
        self.kdim = kdim if kdim is not None else embed_dim
        self.vdim = vdim if vdim is not None else embed_dim
        self._qkv_same_embed_dim = self.kdim == embed_dim and self.vdim == embed_dim

        self.num_heads = num_heads
        self.dropout = dropout
        self.head_dim = embed_dim // num_heads
        assert self.head_dim * num_heads == self.embed_dim, "embed_dim must be divisible by num_heads"

        if self._qkv_same_embed_dim is False:
            self.q_proj_weight = Parameter(torch.Tensor(embed_dim, embed_dim))
            self.k_proj_weight = Parameter(torch.Tensor(embed_dim, self.kdim))
            self.v_proj_weight = Parameter(torch.Tensor(embed_dim, self.vdim))
            self.register_parameter('in_proj_weight', None)
        else:
            self.in_proj_weight = Parameter(torch.empty(3 * embed_dim, embed_dim))
            self.register_parameter('q_proj_weight', None)
            self.register_parameter('k_proj_weight', None)
            self.register_parameter('v_proj_weight', None)#WQ WK WV初始化 可学习的矩阵

        if bias:
            self.in_proj_bias = Parameter(torch.empty(3 * embed_dim))#bias投影
        else:
            self.register_parameter('in_proj_bias', None)
        self.out_proj = Linear(embed_dim, embed_dim, bias=bias)

        if add_bias_kv:
            self.bias_k = Parameter(torch.empty(1, 1, embed_dim))
            self.bias_v = Parameter(torch.empty(1, 1, embed_dim))
        else:
            self.bias_k = self.bias_v = None

        self.add_zero_attn = add_zero_attn

        self._reset_parameters()

    def _reset_parameters(self):
        if self._qkv_same_embed_dim:
            xavier_uniform_(self.in_proj_weight)
        else:
            xavier_uniform_(self.q_proj_weight)
            xavier_uniform_(self.k_proj_weight)
            xavier_uniform_(self.v_proj_weight)

        if self.in_proj_bias is not None:
            constant_(self.in_proj_bias, 0.)
            constant_(self.out_proj.bias, 0.)
        if self.bias_k is not None:
            xavier_normal_(self.bias_k)
        if self.bias_v is not None:
            xavier_normal_(self.bias_v)

    def __setstate__(self, state):
        super(DAGMultiheadAttention, self).__setstate__(state)

        # Support loading old MultiheadAttention checkpoints generated by v1.1.0
        if 'self._qkv_same_embed_dim' not in self.__dict__:
            self._qkv_same_embed_dim = True

    def forward(self, query, key, value, key_padding_mask=None,
                need_weights=True, attn_mask=None):
        # type: (Tensor, Tensor, Tensor, Optional[Tensor], bool, Optional[Tensor]) -> Tuple[Tensor, Optional[Tensor]]
       
        if not self._qkv_same_embed_dim:
            return DAGmulti_head_attention_forward(
                query, key, value, self.embed_dim, self.num_heads,
                self.in_proj_weight, self.in_proj_bias,
                self.bias_k, self.bias_v, self.add_zero_attn,
                self.dropout, self.out_proj.weight, self.out_proj.bias,
                training=self.training,
                key_padding_mask=key_padding_mask, need_weights=need_weights,
                attn_mask=attn_mask, use_separate_proj_weight=True,
                q_proj_weight=self.q_proj_weight, k_proj_weight=self.k_proj_weight,
                v_proj_weight=self.v_proj_weight)
        else:
            return DAGmulti_head_attention_forward(
                query, key, value, self.embed_dim, self.num_heads,
                self.in_proj_weight, self.in_proj_bias,
                self.bias_k, self.bias_v, self.add_zero_attn,
                self.dropout, self.out_proj.weight, self.out_proj.bias,
                training=self.training,
                key_padding_mask=key_padding_mask, need_weights=need_weights,
                attn_mask=attn_mask)

def create_attn_mask(tensor,num_heads):
    mask=torch.zeros((tensor.shape[0]*num_heads),tensor.shape[1],tensor.shape[2])
    for x in range(0,mask.shape[0],num_heads):
        mask[x:x+num_heads]=tensor[x//num_heads]
    return mask.cuda()
def DAGmulti_head_attention_forward(query,                           # type: Tensor
                                 key,                             # type: Tensor
                                 value,                           # type: Tensor
                                 embed_dim_to_check,              # type: int
                                 num_heads,                       # type: int
                                 in_proj_weight,                  # type: Tensor
                                 in_proj_bias,                    # type: Tensor
                                 bias_k,                          # type: Optional[Tensor]
                                 bias_v,                          # type: Optional[Tensor]
                                 add_zero_attn,                   # type: bool
                                 dropout_p,                       # type: float
                                 out_proj_weight,                 # type: Tensor
                                 out_proj_bias,                   # type: Tensor
                                 training=True,                   # type: bool
                                 key_padding_mask=None,           # type: Optional[Tensor]
                                 need_weights=True,               # type: bool
                                 attn_mask=None,                  # type: Optional[Tensor]
                                 use_separate_proj_weight=False,  # type: bool
                                 q_proj_weight=None,              # type: Optional[Tensor]
                                 k_proj_weight=None,              # type: Optional[Tensor]
                                 v_proj_weight=None,              # type: Optional[Tensor]
                                 static_k=None,                   # type: Optional[Tensor]
                                 static_v=None                    # type: Optional[Tensor]
                                 ):
    # type: (...) -> Tuple[Tensor, Optional[Tensor]]
    

    tgt_len, bsz, embed_dim = query.size()
    assert embed_dim == embed_dim_to_check
    assert key.size() == value.size()

    head_dim = embed_dim // num_heads
    assert head_dim * num_heads == embed_dim, "embed_dim must be divisible by num_heads"
    scaling = float(head_dim) ** -0.5

    if not use_separate_proj_weight:
        if torch.equal(query, key) and torch.equal(key, value):
            # self-attention
            q, k, v = linear(query, in_proj_weight, in_proj_bias).chunk(3, dim=-1)

        elif torch.equal(key, value):
            # encoder-decoder attention
            # This is inline in_proj function with in_proj_weight and in_proj_bias
            _b = in_proj_bias
            _start = 0
            _end = embed_dim
            _w = in_proj_weight[_start:_end, :]
            if _b is not None:
                _b = _b[_start:_end]
            q = linear(query, _w, _b)

            if key is None:
                assert value is None
                k = None
                v = None
            else:

                # This is inline in_proj function with in_proj_weight and in_proj_bias
                _b = in_proj_bias
                _start = embed_dim
                _end = None
                _w = in_proj_weight[_start:, :]
                if _b is not None:
                    _b = _b[_start:]
                k, v = linear(key, _w, _b).chunk(2, dim=-1)

        else:
            # This is inline in_proj function with in_proj_weight and in_proj_bias
            _b = in_proj_bias
            _start = 0
            _end = embed_dim
            _w = in_proj_weight[_start:_end, :]
            if _b is not None:
                _b = _b[_start:_end]
            q = linear(query, _w, _b)

            # This is inline in_proj function with in_proj_weight and in_proj_bias
            _b = in_proj_bias
            _start = embed_dim
            _end = embed_dim * 2
            _w = in_proj_weight[_start:_end, :]
            if _b is not None:
                _b = _b[_start:_end]
            k = linear(key, _w, _b)

            # This is inline in_proj function with in_proj_weight and in_proj_bias
            _b = in_proj_bias
            _start = embed_dim * 2
            _end = None
            _w = in_proj_weight[_start:, :]
            if _b is not None:
                _b = _b[_start:]
            v = linear(value, _w, _b)
    else:
        q_proj_weight_non_opt = torch.jit._unwrap_optional(q_proj_weight)
        len1, len2 = q_proj_weight_non_opt.size()
        assert len1 == embed_dim and len2 == query.size(-1)

        k_proj_weight_non_opt = torch.jit._unwrap_optional(k_proj_weight)
        len1, len2 = k_proj_weight_non_opt.size()
        assert len1 == embed_dim and len2 == key.size(-1)

        v_proj_weight_non_opt = torch.jit._unwrap_optional(v_proj_weight)
        len1, len2 = v_proj_weight_non_opt.size()
        assert len1 == embed_dim and len2 == value.size(-1)

        if in_proj_bias is not None:
            q = linear(query, q_proj_weight_non_opt, in_proj_bias[0:embed_dim])
            k = linear(key, k_proj_weight_non_opt, in_proj_bias[embed_dim:(embed_dim * 2)])
            v = linear(value, v_proj_weight_non_opt, in_proj_bias[(embed_dim * 2):])
        else:
            q = linear(query, q_proj_weight_non_opt, in_proj_bias)
            k = linear(key, k_proj_weight_non_opt, in_proj_bias)
            v = linear(value, v_proj_weight_non_opt, in_proj_bias)
    q = q * scaling

    if bias_k is not None and bias_v is not None:
        if static_k is None and static_v is None:
            k = torch.cat([k, bias_k.repeat(1, bsz, 1)])
            v = torch.cat([v, bias_v.repeat(1, bsz, 1)])
            if attn_mask is not None:
                attn_mask = torch.cat([attn_mask,
                                      torch.zeros((attn_mask.size(0), 1),
                                                  dtype=attn_mask.dtype,
                                                  device=attn_mask.device)], dim=1)
            if key_padding_mask is not None:
                key_padding_mask = torch.cat(
                    [key_padding_mask, torch.zeros((key_padding_mask.size(0), 1),
                                                   dtype=key_padding_mask.dtype,
                                                   device=key_padding_mask.device)], dim=1)
        else:
            assert static_k is None, "bias cannot be added to static key."
            assert static_v is None, "bias cannot be added to static value."
    else:
        assert bias_k is None
        assert bias_v is None

    q = q.contiguous().view(tgt_len, bsz * num_heads, head_dim).transpose(0, 1)
    if k is not None:
        k = k.contiguous().view(-1, bsz * num_heads, head_dim).transpose(0, 1)
    if v is not None:
        v = v.contiguous().view(-1, bsz * num_heads, head_dim).transpose(0, 1)

    if static_k is not None:
        assert static_k.size(0) == bsz * num_heads
        assert static_k.size(2) == head_dim
        k = static_k

    if static_v is not None:
        assert static_v.size(0) == bsz * num_heads
        assert static_v.size(2) == head_dim
        v = static_v

    src_len = k.size(1)

    if key_padding_mask is not None:
        assert key_padding_mask.size(0) == bsz
        assert key_padding_mask.size(1) == src_len

    if add_zero_attn:
        src_len += 1
        k = torch.cat([k, torch.zeros((k.size(0), 1) + k.size()[2:], dtype=k.dtype, device=k.device)], dim=1)
        v = torch.cat([v, torch.zeros((v.size(0), 1) + v.size()[2:], dtype=v.dtype, device=v.device)], dim=1)
        if attn_mask is not None:
            attn_mask = torch.cat([attn_mask, torch.zeros((attn_mask.size(0), 1),
                                                          dtype=attn_mask.dtype,
                                                          device=attn_mask.device)], dim=1)
        if key_padding_mask is not None:
            key_padding_mask = torch.cat(
                [key_padding_mask, torch.zeros((key_padding_mask.size(0), 1),
                                               dtype=key_padding_mask.dtype,
                                               device=key_padding_mask.device)], dim=1)

    attn_output_weights = torch.bmm(q, k.transpose(1, 2))
    assert list(attn_output_weights.size()) == [bsz * num_heads, tgt_len, src_len]

    if attn_mask is not None:
        attn_mask = create_attn_mask(attn_mask,num_heads)
        attn_output_weights += attn_mask

    if key_padding_mask is not None:
        attn_output_weights = attn_output_weights.view(bsz, num_heads, tgt_len, src_len)
        attn_output_weights = attn_output_weights.masked_fill(
            key_padding_mask.unsqueeze(1).unsqueeze(2),
            float('-inf'),
        )
        attn_output_weights = attn_output_weights.view(bsz * num_heads, tgt_len, src_len)

    attn_output_weights = softmax(
        attn_output_weights, dim=-1)
    attn_output_weights = dropout(attn_output_weights, p=dropout_p, training=training)

    attn_output = torch.bmm(attn_output_weights, v)
    assert list(attn_output.size()) == [bsz * num_heads, tgt_len, head_dim]
    attn_output = attn_output.transpose(0, 1).contiguous().view(tgt_len, bsz, embed_dim)
    attn_output = linear(attn_output, out_proj_weight, out_proj_bias)

    if need_weights:
        # average attention weights over heads
        attn_output_weights = attn_output_weights.view(bsz, num_heads, tgt_len, src_len)
        return attn_output, attn_output_weights.sum(dim=1) / num_heads
    else:
        return attn_output, None


class DAGTransformerEncoderLayer(nn.Module):
    
    

    def __init__(self, d_model, nhead, dim_feedforward=2048, dropout=0.1, activation="relu"):
        super(DAGTransformerEncoderLayer, self).__init__()
        self.self_attn = DAGMultiheadAttention(d_model, nhead, dropout=dropout)
        # Implementation of Feedforward model
        self.linear1 = Linear(d_model, dim_feedforward)
        self.dropout = Dropout(dropout)
        self.linear2 = Linear(dim_feedforward, d_model)

        self.norm1 = LayerNorm(d_model)
        self.norm2 = LayerNorm(d_model)
        self.dropout1 = Dropout(dropout)
        self.dropout2 = Dropout(dropout)

        self.activation = _get_activation_fn(activation)

    def forward(self, src, src_mask=None, src_key_padding_mask=None):
        
        src2 = self.self_attn(src, src, src, attn_mask=src_mask,
                              key_padding_mask=src_key_padding_mask)[0]
        src = src + self.dropout1(src2)
        src = self.norm1(src)
        if hasattr(self, "activation"):
            src2 = self.linear2(self.dropout(self.activation(self.linear1(src))))
        else:  # for backward compatibility
            src2 = self.linear2(self.dropout(F.relu(self.linear1(src))))
        src = src + self.dropout2(src2)
        src = self.norm2(src)
        return src

class Model(nn.Module):
    def __init__(self, config):
        super(Model, self).__init__()
        self.resnet1=resnet(config.embed,config.txdim,1,config.dropout)
        self.resnet2=resnet(config.embed,config.txdim,1,config.dropout)
        self.conv=nn.Conv1d(in_channels=config.embed,out_channels=config.txdim,kernel_size=1)
        self.conv2=nn.Conv1d(in_channels=config.txdim,out_channels=config.embed,kernel_size=1)
        self.res1=nn.ModuleList([
            copy.deepcopy(self.resnet1)
            for _ in range(config.res_num_layer)
            
        ])
        self.res2=nn.ModuleList([
            copy.deepcopy(self.resnet2)
            for _ in range(config.res_num_layer)
        ])
        self.encoder = DAGTransformerEncoderLayer(config.txdim, config.num_head, config.hidden, config.dropout)
        self.encoders = nn.ModuleList([
            copy.deepcopy(self.encoder)
            # Encoder(config.dim_model, config.num_head, config.hidden, config.dropout)
            for _ in range(config.num_encoder)])
        self.avgpool=nn.AdaptiveAvgPool2d((config.dim_model,1))
        self.fc1 = nn.Linear(config.dim_model, config.num_classes)
        self.dropout=nn.Dropout(config.dropout)
        

    def forward(self,data,pos,attn_mask):
        out = data
        
        out1= pos
        attn_mask=attn_mask
        
        out=out.permute(0,2,1)#shape=[batch_size,embed,src_len]
        out1=out1.permute(0,2,1)
        
        for resnet in self.res1:
            out = resnet(out)
        for resnet in self.res2:
            out1=resnet(out1)

        out=self.conv(out+out1)
        out=out.permute(2,0,1)
        for encoder in self.encoders:
            out = encoder(out.cuda(),src_mask=attn_mask)
        out=out.permute(1,2,0)
        out=self.conv2(out)

        out=self.avgpool(out)
        
        out = out.squeeze(-1)
        
        out = self.fc1(out)
        return out

    
    
class resnet(nn.Module):
    def __init__(self,embed,txdim,kernel_size,dropout):
        super(resnet,self).__init__()
        self.conv1=nn.Conv1d(in_channels=embed,out_channels=txdim,kernel_size=kernel_size)
        self.conv2=nn.Conv1d(in_channels=txdim,out_channels=embed,kernel_size=kernel_size)
        self.dropout=nn.Dropout(dropout)
        #self.bn1=nn.BatchNorm1d(num_features=txdim)
        #self.bn2=nn.BatchNorm1d(num_features=embed)
    def forward(self,x):
        out=x
        #out = F.relu(self.bn1(self.conv1(x)))
        out=self.conv1(x)
        #out = self.bn2(self.conv2(out))
        out=self.conv2(out)
        #out=self.dropout(out)
        out+=x
        #out = F.relu(out)        
        return out

import torch
from torch.optim.lr_scheduler import _LRScheduler
 
 
class WarmUpLR(_LRScheduler):
    """warmup_training learning rate scheduler
    Args:
        optimizer: optimzier(e.g. SGD)
        total_iters: totoal_iters of warmup phase
    """
    def __init__(self, optimizer, total_iters, last_epoch=-1):
        
        self.total_iters = total_iters
        super().__init__(optimizer, last_epoch)
 
    def get_lr(self):
        """we will use the first m batches, and set the learning
        rate to base_lr * m / total_iters
        """
        return [base_lr * self.last_epoch / (self.total_iters + 1e-8) for base_lr in self.base_lrs]
class downLR(_LRScheduler):
    """warmup_training learning rate scheduler
    Args:
        optimizer: optimzier(e.g. SGD)
        total_iters: totoal_iters of warmup phase
    """
    def __init__(self, optimizer, total_iters, last_epoch=-1):
        
        self.total_iters = total_iters
        super().__init__(optimizer, last_epoch)
 
    def get_lr(self):
        """we will use the first m batches, and set the learning
        rate to base_lr * m / total_iters
        """
        return [base_lr * (self.total_iters-self.last_epoch)/ (self.total_iters + 1e-8) for base_lr in self.base_lrs]


from tqdm import tqdm
def train(config, model, train_iter, dev_iter, test_iter):
    start_time = time.time()
    model.train()
    '''optimizer = torch.optim.Adam(model.parameters(), lr=config.learning_rate)
    warmup_epoch = 5
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, config.num_epochs - warmup_epoch)
    
    iter_per_epoch = len(train_iter)
    warmup_scheduler = WarmUpLR(optimizer, warmup_epoch)
'''
    optimizer = torch.optim.Adam(model.parameters(), lr=config.learning_rate)
    warmup_epoch = config.num_epochs/2
    iter_per_epoch = len(train_iter)
    #scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, (config.num_epochs - warmup_epoch))
    scheduler=downLR(optimizer, (config.num_epochs-warmup_epoch)*iter_per_epoch)
    
    warmup_scheduler = WarmUpLR(optimizer, warmup_epoch*iter_per_epoch)
    
    total_batch = 0  
    dev_best_loss = float('inf')
    dev_best_acc = float(0)
    test_best_acc=float(0)
    writer = SummaryWriter(log_dir=config.log_path + '/' + time.strftime('%m-%d_%H.%M', time.localtime()))
    lrlist=np.zeros((config.num_epochs,2))
    for epoch in range(config.num_epochs):
        
        
        loss_total = 0
        print('Epoch [{}/{}]'.format(epoch + 1, config.num_epochs))
        lrlist[epoch][0]=epoch
        predic_all = torch.LongTensor([]).to(config.device)
        true_all = torch.LongTensor([]).to(config.device)
        if(epoch>=warmup_epoch):
            
            learn_rate = scheduler.get_lr()[0]
            print("Learn_rate:%s" % learn_rate)
            lrlist[epoch][1]=learn_rate
        else:
            learn_rate = warmup_scheduler.get_lr()[0]
            lrlist[epoch][0]=learn_rate
            print("Learn_rate:%s" % learn_rate)
        
        
        for i, (trains, labels, poss, masks) in tqdm(enumerate(train_iter)):
            
            outputs = model(trains,poss,masks)
            model.zero_grad()
            loss = F.cross_entropy(outputs, labels)
            loss.backward()
           
            
            optimizer.step()
            if(epoch<warmup_epoch):
                warmup_scheduler.step()
            else:
                scheduler.step()
            total_batch += 1
            loss_total+=loss
            
        
        
        
            true = labels.data
            predic = torch.max(outputs.data, 1)[1]
            predic_all=torch.cat((predic_all, predic),0)
            true_all=torch.cat((true_all, true),0)
            
        train_acc = get_accuracy(true_all, predic_all)
        lossoutput=loss_total/len(train_iter)
                
                
        dev_acc, dev_loss = evaluate(config, model, dev_iter)
        if dev_loss < dev_best_loss:
            dev_best_loss = dev_loss
            
            improve = '*'
                    
        else:
            improve = ''
        if dev_acc > dev_best_acc:
            dev_best_acc = dev_acc
        test_acc, test_loss = evaluate(config, model, test_iter)
                
        if test_acc > test_best_acc:
            test_best_acc = test_acc
            #torch.save(model.state_dict(), './best.ckpt')
        time_dif = get_time_dif(start_time)
        msg = 'Iter: {0:>6},  Train Loss: {1:>5.2},  Train Acc: {2:>6.2%},  Val Loss: {3:>5.2},  Val Acc: {4:>6.2%}, Test Loss: {5:>5.2}, Test Acc: {6:>6.2%},Time: {7} {8}'
        print(msg.format(total_batch, loss.item(), train_acc, dev_loss, dev_acc, test_loss,test_acc,time_dif, improve))
        writer.add_scalar("loss/train", lossoutput.item(), total_batch)
        writer.add_scalar("loss/dev", dev_loss, total_batch)
        writer.add_scalar("loss/test", test_loss, total_batch)
        writer.add_scalar("acc/train", train_acc,  total_batch)
        writer.add_scalar("acc/dev", dev_acc, total_batch)
        writer.add_scalar("acc/test", test_acc, total_batch)
        model.train()
                
            
           
        print("dev best acc=",dev_best_acc)
        print("test best acc=",test_best_acc)
        
    writer.close()
    test(config, model, test_iter)


def test(config, model, test_iter):
    # test
    #model.load_state_dict(torch.load(config.save_path))
    model.eval()
    start_time = time.time()
    test_acc, test_loss, test_confusion = evaluate(config, model, test_iter, test=True)
    msg = 'Test Loss: {0:>5.2},  Test Acc: {1:>6.2%}'
    print(msg.format(test_loss, test_acc))
    print("Confusion Matrix...")
    print(test_confusion)
    time_dif = get_time_dif(start_time)
    print("Time usage:", time_dif)


def evaluate(config, model, data_iter, test=False):
    model.eval()
    loss_total = 0
    predict_all = torch.LongTensor([]).to(config.device)
    labels_all = torch.LongTensor([]).to(config.device)
    with torch.no_grad():
        for texts, labels, poss, masks in data_iter:
            outputs = model(texts,poss,masks)
            loss = F.cross_entropy(outputs, labels)
            loss_total += loss
            labels = labels.data
            predic = torch.max(outputs.data, 1)[1]
            labels_all = torch.cat((labels_all, labels),0)
            predict_all = torch.cat((predict_all, predic),0)
    
    acc = get_accuracy(labels_all, predict_all)
    if test:
        #report = metrics.classification_report(labels_all, predict_all, target_names=config.class_list, digits=4)
        confusion = metrics.confusion_matrix(labels_all.cpu().numpy(), predict_all.cpu().numpy())
        return acc, loss_total / len(data_iter), confusion
    return acc, loss_total / len(data_iter)
start_time = time.time()
print("Loading data...")

def get_accuracy(y_true,y_pred):
    y_true,y_pred=y_true.cpu().numpy(), y_pred.cpu().numpy()
    return metrics.accuracy_score(y_true,y_pred)

time_dif = get_time_dif(start_time)
print("Time usage:", time_dif)
seed = 77
torch.manual_seed(seed)            
torch.cuda.manual_seed(seed)       
torch.cuda.manual_seed_all(seed)   
model = Model(config).to(config.device)
train(config, model, train_iter, dev_iter, test_iter)

/home/luvyf/anaconda3/envs/pydl37/lib/python3.7/site-packages/ipykernel_launcher.py:149: RuntimeWarning: invalid value encountered in true_divide


Loading data...
Time usage: 0:00:00
Epoch [1/1000]
Learn_rate:0.0


108it [00:14,  7.26it/s]


Iter:    108,  Train Loss:   1.1,  Train Acc: 40.75%,  Val Loss:   1.0,  Val Acc: 41.65%, Test Loss:   1.0, Test Acc: 42.18%,Time: 0:00:18 *
dev best acc= 0.41648238242665486
test best acc= 0.4218310900754326
Epoch [2/1000]
Learn_rate:1.9999999999996299e-07


108it [00:14,  7.23it/s]


...

Iter:  52488,  Train Loss:  0.22,  Train Acc: 91.56%,  Val Loss:  0.28,  Val Acc: 90.98%, Test Loss:  0.29, Test Acc: 90.16%,Time: 2:29:11 
dev best acc= 0.9131652661064426
test best acc= 0.9077059606567076
Epoch [487/1000]
Learn_rate:9.7199999999982e-05


108it [00:15,  7.06it/s]


Iter:  52596,  Train Loss:   0.2,  Train Acc: 91.61%,  Val Loss:  0.28,  Val Acc: 91.07%, Test Loss:  0.29, Test Acc: 90.39%,Time: 2:29:30 
dev best acc= 0.9131652661064426
test best acc= 0.9077059606567076
Epoch [488/1000]
Learn_rate:9.739999999998197e-05


34it [00:04,  7.09it/s]


KeyboardInterrupt: 